# 2.9 部署模型
## 🚄 前言
在经过模型微调和评测后，开发工作已经接近完成。本节课程将介绍最后一部分的工作：将你的模型部署在计算资源上，使其成为一个可访问的应用服务。

## 🍁 课程目标
本节课程的主要内容包括：
- 学习基于vLLM部署本地模型
- 了解阿里云的云端部署平台，学习常见的部署方案

## 📖 课程目录

- 1.&nbsp; 本地部署模型
- 2.&nbsp; 云端部署模型

## 🚀 1. 本地部署模型
本地部署模型主要包含下载模型、编写模型加载代码和发布为支持API访问的应用服务这三个步骤。这个过程通常伴随较高的人工部署成本，vLLM可以用来简化这一流程。它是一个专为大模型推理设计的开源框架，能够以简单的命令行参数快速部署模型，并通过内存优化和缓存策略提升推理速度和支持高并发请求。

本课程将使用vLLM加载模型并启动模型服务。该服务提供的HTTP接口兼容OpenAI API，可以通过调用HTTP接口，如/v1/chat/completions，让你快速体验大模型的推理能力。

### 步骤1：环境准备

本课程实验环境需与第2.8微调章节保持一致，确保在GPU环境下执行模型部署操作。 

如已按课程目录顺序学习，请继续使用第2.8章节启动的PAI-DSW实例；若单独学习本章节，则按第2.8章节环境准备进行配置即可。

请在指定目录下打开Terminal终端窗口。

在本课程的目录下，**即/mnt/workspace/aliyun_acp_learning/大模型ACP认证教程/p2_构造大模型问答系统**，打开一个新的终端窗口。

<img src="https://img.alicdn.com/imgextra/i2/O1CN01rZsIL71Zfj8TpX2Wq_!!6000000003222-2-tps-1392-1334.png" width="400">


你可以在终端窗口中输入`pwd`命令查看终端窗口所在的目录位置。

### 步骤2：使用vLLM部署模型并启动服务

**步骤2.1：部署开源模型**

这里选择在[ModelScope模型库](https://modelscope.cn/models)中下载**Qwen2.5-1.5B-Instruct**模型进行部署。当然，也可以在[HuggingFace模型库](https://huggingface.co/models)下载开源模型进行部署，暂不在此演示。

首先，下载模型文件到本地。

In [ ]:
!mkdir -p ./model/qwen2_5-1_5b-instruct
!modelscope download --model qwen/Qwen2.5-1.5B-Instruct --local_dir './model/qwen2_5-1_5b-instruct'

下载成功后，模型文件会保存在./model/qwen2_5-1_5b-instruct文件夹下。

<img src="https://img.alicdn.com/imgextra/i4/O1CN010iGHeL1REEQvGxUXD_!!6000000002079-0-tps-770-166.jpg" width="400">

接着，在终端窗口执行**vllm命令**启动一个模型服务。

<style>
    table {
      width: 80%;
      margin: 20px; /* Center the table */
      border-collapse: collapse; /* Collapse borders for a cleaner look */
      font-family: sans-serif; 
    }

    th, td {
      padding: 10px;
      text-align: left;
      border: 1px solid #ddd; /* Light gray border */
    }

    th {
      background-color: #f2f2f2; /* Light gray background for header */
      font-weight: bold;
    }

    tr:nth-child(even) { /* Zebra striping */
      background-color: #f9f9f9;
    }

    tr:hover { /* Highlight row on hover */
      background-color: #e0f2ff; /* Light blue */
    }
</style>
<table width="90%">
<tbody>
<tr>
<td>

```bash
vllm serve "./model/qwen2_5-1_5b-instruct" --load-format "safetensors" --port 8000
```
</td>
</tr>
</tbody>
</table>

- vllm serve：表示启动模型服务。
- "./model/qwen2_5-1_5b-instruct"：表示加载的模型路径，通常包含模型文件、版本信息等。
- --load-format "safetensors"：指定加载模型时使用的格式。
- --port 8000：指定端口号，如果端口被占用，请切换为其他端口，如8100等。

服务启动成功后，终端窗口会打印 **“NFO:     Uvicorn running on socket ('0.0.0.0', 8000) (Press CTRL+C to quit)”** 信息。

请注意，关闭终端窗口将会立即终止服务。后续的步骤3和步骤4将依赖于该服务进行测试，因此请务必不要关闭此终端窗口。

> 如果你希望在后台持续运行服务而不受终端窗口关闭的影响，可以使用这条命令。
> ```bash
> # 后台运行服务，且服务的运行日志存储到vllm.log
> nohup vllm serve "./model/qwen2_5-1_5b-instruct" --load-format "safetensors" --port 8000 > vllm.log 2>&1 &
> ```

**步骤2.2：部署微调模型（可选）**

第2.8章节的微调模型默认放在**output**目录。下面示例选择微调后的merge模型进行部署。请打开一个新的终端窗口执行vllm命令。

<style>
    table {
      width: 80%;
      margin: 20px; /* Center the table */
      border-collapse: collapse; /* Collapse borders for a cleaner look */
      font-family: sans-serif; 
    }

    th, td {
      padding: 10px;
      text-align: left;
      border: 1px solid #ddd; /* Light gray border */
    }

    th {
      background-color: #f2f2f2; /* Light gray background for header */
      font-weight: bold;
    }

    tr:nth-child(even) { /* Zebra striping */
      background-color: #f9f9f9;
    }

    tr:hover { /* Highlight row on hover */
      background-color: #e0f2ff; /* Light blue */
    }
</style>
<table width="90%">
<tbody>
<tr>
<td>

```bash
vllm serve "./output/qwen2_5-1_5b-instruct/v0-202xxxxx-xxxxxx/checkpoint-xxx-merged" --load-format "safetensors" --port 8001
```
</td>
</tr>
</tbody>
</table>

- "./output/qwen2_5-1_5b-instruct/v0-202xxxxx-xxxxxx/checkpoint-xxx-merged"：替换为真实的微调模型路径。
- --port 8001：设置与步骤2.1中不同的端口号，避免端口占用。

### 步骤3：测试服务是否正常运行

vLLM 支持启动兼容OpenAI API的本地服务器，即按照OpenAI API标准返回结果。

通过cURL发送HTTP请求，测试步骤2.1部署的**Qwen2.5-1.5B-Instruct**模型服务是否能够正常响应。如果使用微调模型服务，请确保将请求的URL端口号从8000修改为8001。

In [ ]:
%%bash
curl -X POST http://localhost:8000/v1/chat/completions \
    -H "Content-Type: application/json" \
    -d '{
        "model": "./model/qwen2_5-1_5b-instruct",
        "messages": [
            {"role": "system", "content": "你是一个帮助助手。"},
            {"role": "user", "content": "请告诉我2008年北京奥运会，中国队总共获得了多少枚金牌？"}
        ]
    }'

上述接口正常响应即表示服务正常运行。

此外，还兼容了 /v1/models 接口，支持查看部署的模型列表。更多信息请查看[vLLM兼容的OpenAI API](https://docs.vllm.ai/en/latest/serving/openai_compatible_server.html#api-reference)。

In [ ]:
%%bash
curl -X GET http://localhost:8000/v1/models

### 步骤4：评估服务性能

为了评估本地部署的模型服务性能，这里使用一个简单的HTTP性能测试工具**wrk**来快速模拟压测请求并生成报告。下面以压测 **POST /v1/chat/completions** 接口为例，展示服务的相关性能指标。

首先，打开一个新的终端窗口，安装压测工具wrk的依赖包。注意：终端窗口是在步骤1中指定的目录下。

```bash
sudo apt update
sudo apt install wrk
```

接着，准备POST请求所需要的Body数据。数据已放在./resources/2_9/post.lua文件，文件内容如下所示。

```bash
wrk.method = "POST"
wrk.headers["Content-Type"] = "application/json"
wrk.body = [[
    {
       "model": "./model/qwen2_5-1_5b-instruct",
       "messages": [
           {"role": "system", "content": "你是一个帮助助手。"},
           {"role": "user", "content": "请告诉我2008年北京奥运会，中国队总共获得了多少枚金牌？"}
       ]
   }
]]
```

然后，在终端窗口执行wrk压测命令，分别设置chat接口的并发量(-c)为1和10，压测时间(-d)均为10s，观察两个实验的压测结果。

```bash
wrk -t1 -c1 -d10s -s ./resources/2_9/post.lua http://localhost:8000/v1/chat/completions

wrk -t1 -c10 -d10s -s ./resources/2_9/post.lua http://localhost:8000/v1/chat/completions
```

wrk压测结果如下所示。

<img src="https://img.alicdn.com/imgextra/i3/O1CN01ybO7TU1X6LJ12FYdV_!!6000000002874-2-tps-1452-322.png" width="500" height="150">
<img src="https://img.alicdn.com/imgextra/i2/O1CN01bberC61txr86CpFjU_!!6000000005969-2-tps-1472-362.png" width="500" height="150">


根据压测结果可见，随着并发量增加（1 -> 10），QPS提升了约6倍（3.30 -> 20.08），平均延迟增加了约30%（324.61ms -> 426.84ms）。特别地是，第二个压测实验中出现了2个超时错误，表明在并发量较高情况下，服务器的负载超过了其处理能力。由此可见，基于vLLM本地部署的模型在应对并发量提升时能够承载一定的负载，但仍存在性能瓶颈。

## ☁2. 云端部署模型

上述压测实验结果显示，由于本地设备算力有限，本地部署的模型服务无法满足低延迟和高并发的需求。针对这类线上业务，可以考虑云端部署。

下面先来看看本地部署和云端部署的特点对比。

| 特点             | 本地部署                          | 云端部署                          |
|------------------|----------------------------------|-----------------------------------|
| 延迟性           | 低延迟，因为数据处理在本地进行  | 较高延迟，数据需要传输到云端进行处理 |
| 灵活性           | 灵活性较低，资源受限于本地硬件   | 灵活性较高，可根据需求动态调整资源 |
| 稳定可靠         | 依赖于本地硬件和网络的稳定性     | 通常更稳定，云服务提供商提供冗余和备份 |
| 安全             | 数据在本地存储，安全性较高，但管理复杂 | 数据存储在云端，受供应商安全措施保护 |

由上可知，相比本地部署，云端部署提供了高可扩展性、灵活的资源管理以及稳定的服务，从而能够快速适应业务变化。

此外，线上模型服务还需要解决模型管理问题，如上线与下线、扩缩容和推理加速策略等，因此，云上部署是更优的选择。

下面将介绍四种常见的阿里云部署平台，支持不同的模型部署需求，帮助你快速搭建模型服务。

- [云服务器ECS](https://help.aliyun.com/zh/egs/what-is-elastic-gpu-service?spm=a2c4g.11186623.help-menu-155040.d_0_1_1.1f6fda88BUxmmW)：提供底层CPU或GPU服务器，自主控制性强，支持从零搭建部署环境。针对常见的大模型，包括Llama模型、ChatGLM模型、百川模型、通义千问模型及其微调模型，推出DeepGPU-LLM工具加速模型推理。ECS部署适合有运维团队、需要自主管控和长时间稳定运行的业务。

- [函数计算FC](https://help.aliyun.com/zh/functioncompute/fc-3-0/product-overview/what-is-function-compute?spm=a2c4g.11186623.help-menu-2508973.d_0_2_0.4e07511e12fCMr)：提供Serverless和Serverless GPU服务，让你专注于核心业务，免去运维负担。函数计算按调用计费，支持常驻任务，尤其适合计算资源要求高的临时任务。但是，需要接受任务之间的服务冷启动延迟。

- [人工智能平台PAI-EAS](https://help.aliyun.com/zh/pai/user-guide/overview-2?spm=a2c4g.11186623.help-menu-30347.d_2_3_9_0.41427febaJGiYN#a6d6dca142s5f) ：针对实时推理和AI模型服务需求高的场景，PAI-EAS 提供自动扩展和版本管理，适合对并发和服务持续性有严格要求的业务。

- [大模型服务平台百炼](https://help.aliyun.com/zh/model-studio/getting-started/what-is-model-studio?spm=a2c4g.11186623.help-menu-2400256.d_0_0.44f7716e0UV3gO&scm=20140722.H_2579562._.OR_help-V_1)：模型部署成本低，适合闭源或托管模型服务场景。百炼的独占实例管理支持资源动态调整，能高效处理闭源模型的高并发访问。





### 2.1. 云服务器ECS

ECS部署模型可控性高、可扩展性强、能支撑复杂的业务场景，它的主要部署步骤如下图所示。

<img src="https://img.alicdn.com/imgextra/i1/O1CN01HjJQcq22OR2dxkfZO_!!6000000007110-0-tps-1402-164.jpg" width="700">

- ECS选型：按需求选择 CPU 或 GPU 类型的服务器，GPU可选择 Tesla T4、V100、A100 等。
- 配置环境：按需配置服务运行的环境，如Docker镜像环境、第三方依赖、安全组环境等。
- 部署服务：推荐使用镜像部署，并在镜像中基于vLLM启动服务，该服务运行在云服务器的本地环境。
- 提供服务：实验测试可用本地IP访问；正式对外服务需申请公网域名并进行域名备案以获得可公开访问的URL。

你可以参考[使用vLLM容器镜像快速构建大语言模型在GPU上的推理环境](https://help.aliyun.com/zh/egs/use-cases/use-a-vllm-container-image-to-run-inference-tasks-on-a-gpu-accelerated-instance)执行具体操作。

如果你要部署的模型是Llama模型、ChatGLM模型、百川模型、通义千问模型及其微调模型，请[安装并使用DeepGPU-LLM进行模型的推理服务](https://help.aliyun.com/zh/egs/developer-reference/install-and-use-deepgpu-llm-for-model-inference?spm=a2c4g.11186623.0.i6)以加速模型推理能力。


### 2.2. 函数计算FC

函数计算FC提供Serverless和Serverless GPU服务，按需扩缩容，极少运维，但GPU服务的冷启动相对时间较长。函数计算控制台的镜像部署步骤如下图所示。

<img src="https://img.alicdn.com/imgextra/i3/O1CN01xxiIod1WONHe7QRtG_!!6000000002778-0-tps-1644-364.jpg" width="800">

- 配置函数应用：按需选择函数应用模板或者配置自定义函数。
- 配置镜像：配置模型所需的镜像环境。
- 从存储服务拉取模型：函数计算不存储模型，模型需要借助存储服务来存储，如NAS、OSS等。
- 一键部署：在镜像中启动模型服务，并部署到函数计算GPU资源中。这个过程由函数计算内部执行，对外则提供一键部署按钮，点击即可完成部署。
- 自动生成公网域名访问服务：函数计算提供自动生成的公网域名，也支持修改为自定义域名，以提供访问服务的入口。

你可以[通过函数计算部署Flux文本生成图像模型](https://help.aliyun.com/zh/functioncompute/fc-3-0/use-cases/use-function-compute-to-deploy-flux-model-to-generate-image-from-text?spm=a2c4g.11186623.help-menu-2508973.d_3_0.6a324d57BwfYxj&scm=20140722.H_2850993._.OR_help-V_1)体验函计算提供的部署能力，更多部署实践可参见[函数计算3.0-实践教程](https://help.aliyun.com/zh/functioncompute/fc-3-0/use-cases/?spm=a2c4g.11186623.help-menu-2508973.d_3.228e493fj6un1Y&scm=20140722.H_2509019._.OR_help-V_1)。

### 2.3. 人工智能平台PAI-EAS

PAI-EAS是一种模型在线服务平台，支持一键部署模型为在线推理服务。它提供的弹性扩缩容、蓝绿部署、资源组管理、版本控制以及资源监控等功能，适用于实时同步推理、近实时异步推理等多种AI推理场景。PAI-EAS支持镜像部署和Processor部署两种方式，镜像部署的主要步骤如下所示。

<img src="https://img.alicdn.com/imgextra/i2/O1CN01cSjKDo1C6FI8IdKh0_!!6000000000031-0-tps-1568-160.jpg" width="750">

- 上传模型及数据：支持上传开源社区或者自定义的模型，除此之外，还可以上传代码、模型预热数据等。
- 配置环境：可以使用PAI预置的官方镜像，也可以使用自定义镜像，同时还可能需要安装第三方依赖。
- 部署服务：在镜像中启动vLLM服务。
- 基于EAS-API调用服务：服务启动成功后，EAS会自动生成公网域名访问URL，用于调用服务。

你可以参考[5分钟操作EAS一键部署通义千问模型](https://help.aliyun.com/zh/pai/use-cases/use-pai-eas-to-quickly-deploy-tongyi-qianwen?spm=a2c4g.11186623.0.i0#ba6b53303bb66)来快速部署一个通用模型，立即体验PAI-EAS所提供的模型服务能力。

若你希望部署自定义模型，建议参考[5分钟使用EAS一键部署LLM大语言模型应用](https://help.aliyun.com/zh/pai/use-cases/deploy-llm-in-eas#c1d769ba33kh5)。

### 2.4. 大模型服务平台百炼

百炼平台支持通过页面快速部署模型，同时也允许通过简单的API调用进行部署。整个部署操作比较简单，如下所示。

<img src="https://img.alicdn.com/imgextra/i3/O1CN01dMHbz61gQnXOvvBa2_!!6000000004137-0-tps-1192-166.jpg" width="600">

- 选择模型：百炼平台仅支持部署预置模型及经过微调后的百炼模型，不支持部署自定义模型。
- 一键部署模型：控制台上支持一键部署模型，也可以通过API快速部署。
- 基于百炼生态使用模型：部署后的模型可无缝集成百炼生态，支持在百炼控制台直接使用，并可通过HTTP和DashScope调用复用百炼API。

模型部署操作可参考[百炼部署模型](https://help.aliyun.com/zh/model-studio/user-guide/model-deployment)文档。

## ✅本节小结

本节课程详细介绍了模型部署的基本方法，你学习了：
- 本地部署模型的实操步骤，即如何将模型部署为一个可访问的模型推理服务，部署模型可以是开源模型或微调后的模型。
- 云端部署模型的平台选型介绍及部署步骤。

通过本课程的学习，你已经掌握了模型部署的基本方法，为构建高性能、可扩展的大模型应用打下了坚实的基础。

接下来，你将学习一些借助大模型辅助内容生产的案例，如翻译、音视频转换、PPT制作等。希望这些案例能启发你开发更多效率工具。

>⚠️ **注意**：完成本节学习后，请及时停止当前的 PAI-DSW GPU 实例，以避免产生额外费用。
> 
>  进入第 3 部分学习前，建议继续使用2_8章节前所创建的 PAI-DSW CPU 实例。


### 扩展阅读

本课程介绍了云端部署，在实际部署中可分为公有云部署和私有云部署。

- **公有云部署**：将模型封装为API供用户调用，类似SaaS模式。这种方式降低了使用门槛，便于集成，但需要确保API稳定和安全。

- **私有云部署**：在企业内部搭建私有云平台，并将模型部署在私有云上。它能提供更高的数据安全性和控制权，支持定制化，但需投入一定的维护成本。

此外，端云协同部署也是一种常见的部署方式。

**端云协同部署**结合云端和边缘部署的优势，部分计算在边缘设备上进行，复杂计算任务则上传到云端处理，能够在保证用户体验的同时处理复杂的计算任务。

端云协同部署适用于需要实时响应但计算资源有限的场景，如乐天派与通义大模型合作打造端侧陪伴智能语音机器人，见下图。这里的“端”指客户端部署的小模型，“云”指云端部署的大模型。这些小模型需要微调以适配不同客户端设备，并与客户端应用在同一个环境中，负责初步处理数据预处理等简单任务，然后将处理后的信息传输给云端，由云端大模型进行深入处理，以快速且高质量响应端侧用户需求。

<img src="https://img.alicdn.com/imgextra/i2/O1CN015VBCB524nk9sYPd8s_!!6000000007436-0-tps-1662-474.jpg" width="650">

在一些特定场景，**嵌入式系统部署**是更为合适的选择，如汽车、机器人和医疗设备等。这种方式将模型部署在硬件平台上，能够实现实时控制和决策，但对模型和硬件的优化要求较高。

因此，在面对实际业务需求时，你需要考虑系统的性能要求、数据隐私和安全性，以及实施的复杂性，从而确保部署方案的高效与可持续性。

## ✉️ 评价反馈
感谢你学习阿里云大模型ACP认证课程，如果你觉得课程有哪里写得好、哪里写得不好，期待你[通过问卷提交评价和反馈](https://survey.aliyun.com/apps/zhiliao/Mo5O9vuie)。

你的批评和鼓励都是我们前进的动力。